# Smart Bot Orchestration - Automated Grid Trading

## 🎯 Overview
Intelligent bot management system that analyzes signals, monitors performance, and automatically deploys/manages grid trading strategies on Hummingbot.

## ✨ Key Features
- **Signal-Based Trading**: Automatically deploys controllers based on EMA trend signals
- **Performance Monitoring**: Tracks P&L, volume, and runtime for each controller
- **Intelligent Management**: Stops underperforming or conflicting controllers
- **Telegram Notifications**: Real-time updates at every step
- **Simplified Deployment**: No complex ID tracking - just one bot name

## 📋 How It Works

### Step 1: Signal Analysis
- Fetches top 5 long and top 5 short signals from MongoDB
- Uses EMA trend feature with configurable intensity thresholds
- Sends signal summary via Telegram

### Step 2: Check Current Bot Status
- Looks for active bot with name `trend_follower_grid`
- If found: displays controllers, P&L, runtime
- Sends current status via Telegram

### Step 3: Analyze & Stop Controllers
- **Stops controllers if:**
  - Runtime > min hours AND P&L < threshold (-1% default)
  - Signal direction changed (long→short or short→long)
- **Keeps controllers if:**
  - Runtime < min hours (protection period)
  - Performing well and signal still valid

### Step 4: Deploy New Controllers
- Identifies opportunities not already covered
- Skips recently stopped markets (no immediate redeployment)
- Creates grid configs and deploys bot
- Sends detailed deployment report via Telegram

### Step 5: Final Summary
- Complete orchestration report via Telegram
- Shows stopped, deployed, and kept controllers
- Management rules applied

## ⚙️ Configuration

| Setting | Default | Description |
|---------|---------|-------------|
| `bot_name` | `trend_follower_grid` | Simple bot name (no IDs) |
| `max_controllers_per_instance` | 10 | Maximum trading pairs |
| `min_runtime_hours` | 3 | Protection period before stopping |
| `min_pnl_pct_to_keep` | -1.0% | P&L threshold |
| `stop_on_opposing_signal` | True | Stop if signal flips |
| `total_amount_quote` | $400 | Capital per grid |
| `leverage` | 20x | Position leverage |
| `take_profit` | 0.16% | Grid take profit |

## 🔄 Typical Workflow

### First Run (No Active Bot)
```
1. Load signals → Telegram notification
2. No bot found → Deploy fresh
3. Create 10 controllers (5 long + 5 short)
4. Deploy bot → Telegram notification
5. Final summary → Telegram notification
```

### Subsequent Runs (Bot Active)
```
1. Load signals → Telegram notification
2. Check bot status → Telegram notification
3. Analyze controllers:
   - Keep: 7 controllers (good performance)
   - Stop: 3 controllers (poor P&L or signal flip)
4. Deploy 3 new controllers (replacement)
5. Final summary → Telegram notification
```

## 📱 Telegram Notifications

You'll receive **4 notifications** per run:

1. **🎯 Trading Signals** - Top opportunities identified
2. **🤖 Current Bot Status** - Runtime, P&L, winners/losers (if bot exists)
3. **🚀 Bot Deployed** - Full breakdown of positions and settings
4. **🎉 Orchestration Complete** - Final summary with all actions taken

## 🚀 Quick Start

1. **Set up environment**:
   - Ensure Hummingbot is running on localhost:8000
   - Configure Telegram bot token in .env
   - Run feature engineering notebook first

2. **Run cells in order**:
   - Each cell is designed to run sequentially
   - Telegram notifications happen automatically

3. **Check results**:
   - Console output shows detailed progress
   - Telegram messages provide mobile-friendly summaries
   - Hummingbot dashboard shows active trading

## ⚠️ Important Notes

- **Bot Name**: Always `trend_follower_grid` (no _1, _2 suffixes)
- **Redeployment**: If bot exists, it will be stopped first, then redeployed
- **Signal Updates**: Run feature engineering notebook before this for fresh signals
- **Capital**: Total exposure = $400 × number of controllers × 20x leverage
- **Protection Period**: Controllers are safe from stopping for first 3 hours

## 🛠️ Troubleshooting

**No signals found?**
- Run feature engineering notebook first
- Check MongoDB connection
- Verify signal intensity thresholds

**Bot won't deploy?**
- Check Hummingbot API is running
- Verify credentials profile exists
- Ensure exchange API keys are configured

**Too many controllers stopped?**
- Increase `min_runtime_hours` for longer protection
- Adjust `min_pnl_pct_to_keep` to be more lenient
- Disable `stop_on_opposing_signal` if signals fluctuate

## 📊 Performance Tracking

Monitor your bot's performance through:
- **Telegram**: Real-time notifications with P&L updates
- **Notebook Output**: Detailed controller breakdown
- **Hummingbot Dashboard**: Live trading view
- **MongoDB**: Historical feature and signal data

In [1]:
# Imports and Setup
from datetime import datetime, timezone, timedelta
from dotenv import load_dotenv
import warnings
import pandas as pd

warnings.filterwarnings("ignore")
load_dotenv()

from hummingbot_api_client import HummingbotAPIClient
from core.features import FeatureStorage
from core.notifiers import NotificationManager, NotificationMessage

# Initialize
storage = FeatureStorage()
notification_manager = NotificationManager()

print("✅ Smart Bot Orchestration initialized")
print(f"🔔 Enabled notifiers: {', '.join(notification_manager.get_enabled_notifiers())}")

✅ Smart Bot Orchestration initialized
🔔 Enabled notifiers: telegram


In [33]:
# Configuration
CONFIG = {
    # Bot instance settings - SIMPLIFIED: bot name = strategy name
    'bot_name': 'trend_follower_grid',  # This is now the actual bot name
    'credentials_profile': 'master_account',
    'connector_name': 'binance_perpetual',
    'hummingbot_host': 'localhost',
    
    # Signal settings
    'feature_name': 'ema_trend',
    'signal_category': 'tf',
    'min_long_intensity': 0.7,
    'min_short_intensity': -0.7,
    'top_n_per_side': 5,
    
    # Management rules
    'max_controllers_per_instance': 10,
    'min_runtime_hours': 4,
    'stop_on_opposing_signal': True,
    'min_pnl_pct_to_keep': -0.3,
    
    # Grid parameters
    'total_amount_quote': '400',
    'leverage': 20,
    'position_mode': 'HEDGE',
    'max_open_orders': 2,
    'max_orders_per_batch': 1,
    'activation_bounds': '0.003',
    'min_spread_between_orders': '0.002',
    'take_profit': '0.0016',
    
    # Grid range multipliers
    'grid_range_multiplier': {
        'long': {'start_offset': -0.25, 'end_offset': 1.25, 'limit_offset': -0.35},
        'short': {'start_offset': -1.25, 'end_offset': 0.25, 'limit_offset': 0.35}
    },
    
    # Execution settings
    'auto_stop_controllers': True,
    'auto_deploy': True,
    'send_telegram_notifications': True,  # Send notifications at each step
}

print("📊 Configuration loaded:")
print(f"  Bot name: {CONFIG['bot_name']}")
print(f"  Max controllers: {CONFIG['max_controllers_per_instance']}")
print(f"  Min runtime: {CONFIG['min_runtime_hours']} hours")
print(f"  Min P&L to keep: {CONFIG['min_pnl_pct_to_keep']}%")
print(f"  Telegram notifications: {CONFIG['send_telegram_notifications']}")

📊 Configuration loaded:
  Bot name: trend_follower_grid
  Max controllers: 10
  Min runtime: 4 hours
  Min P&L to keep: -0.3%
  Telegram notifications: True


In [34]:
# Connect to MongoDB
await storage.connect()
print("✅ Connected to MongoDB feature storage")

✅ Connected to MongoDB feature storage


## 1. Load Current Signals

In [35]:
# Fetch top signals
long_signals = await storage.get_signals(
    category=CONFIG['signal_category'],
    min_value=CONFIG['min_long_intensity'],
    limit=CONFIG['top_n_per_side'] * 2
)

short_signals = await storage.get_signals(
    category=CONFIG['signal_category'],
    max_value=CONFIG['min_short_intensity'],
    limit=CONFIG['top_n_per_side'] * 2
)

# Create signal lookup dict
signal_map = {}
for sig in long_signals:
    signal_map[sig.trading_pair] = {'value': sig.value, 'direction': 'long'}
for sig in short_signals:
    signal_map[sig.trading_pair] = {'value': sig.value, 'direction': 'short'}

print(f"🟢 Found {len(long_signals)} long signals (value > {CONFIG['min_long_intensity']})")
print(f"🔴 Found {len(short_signals)} short signals (value < {CONFIG['min_short_intensity']})")

if long_signals:
    print("\n🟢 Top Long Signals:")
    for i, sig in enumerate(long_signals[:CONFIG['top_n_per_side']], 1):
        print(f"  {i}. {sig.trading_pair:12s} | Intensity: {sig.value:+.3f}")

if short_signals:
    print("\n🔴 Top Short Signals:")
    for i, sig in enumerate(short_signals[:CONFIG['top_n_per_side']], 1):
        print(f"  {i}. {sig.trading_pair:12s} | Intensity: {sig.value:+.3f}")

# Send Telegram notification about signals
if CONFIG['send_telegram_notifications']:
    msg = f"""<b>📊 Signal Analysis Complete</b>

<b>🟢 Long Signals ({len(long_signals[:CONFIG['top_n_per_side']])})</b>
"""
    for i, sig in enumerate(long_signals[:CONFIG['top_n_per_side']], 1):
        msg += f"{i}. <code>{sig.trading_pair:12s}</code> {sig.value:+.3f}\n"
    
    msg += f"\n<b>🔴 Short Signals ({len(short_signals[:CONFIG['top_n_per_side']])})</b>\n"
    for i, sig in enumerate(short_signals[:CONFIG['top_n_per_side']], 1):
        msg += f"{i}. <code>{sig.trading_pair:12s}</code> {sig.value:+.3f}\n"
    
    notification = NotificationMessage(
        title="🎯 Trading Signals",
        message=msg,
        level="info"
    )
    await notification_manager.send_notification(notification)
    print("\n✅ Signal analysis sent to Telegram")

🟢 Found 10 long signals (value > 0.7)
🔴 Found 10 short signals (value < -0.7)

🟢 Top Long Signals:
  1. BEAMX-USDT   | Intensity: +0.873
  2. XTZ-USDT     | Intensity: +0.887
  3. ALGO-USDT    | Intensity: +0.759
  4. MTL-USDT     | Intensity: +0.837
  5. TIA-USDT     | Intensity: +0.933

🔴 Top Short Signals:
  1. TWT-USDT     | Intensity: -0.733
  2. SQD-USDT     | Intensity: -0.744
  3. STO-USDT     | Intensity: -0.786
  4. PTB-USDT     | Intensity: -0.839
  5. BAT-USDT     | Intensity: -0.930

✅ Signal analysis sent to Telegram


## 2. Fetch Active Bots and Controllers

In [38]:
# Fetch active bot status - SIMPLIFIED
bot_controllers = []
running_bot_name = None
bot_deployed_at = None

print("🤖 Fetching active bot status from Hummingbot API...\n")

async with HummingbotAPIClient(base_url=f"http://{CONFIG['hummingbot_host']}:8000") as client:
    try:
        # Check if our bot is running
        active_bots_data = await client.bot_orchestration.get_active_bots_status()
        
        if active_bots_data and active_bots_data.get('status') == 'success':
            bots_data = active_bots_data.get('data', {})
            for bot_name, data in bots_data.items():
                if CONFIG['bot_name'] in bot_name:
                    running_bot_name = bot_name
                    bot_info = bots_data[running_bot_name]

                    print(f"✅ Found active bot: {running_bot_name}")

                    # Get deployment time from bot runs
                    bot_runs = await client.bot_orchestration.get_bot_runs(
                        bot_name=running_bot_name,
                        run_status="CREATED"
                    )

                    if bot_runs.get('status') == 'success' and bot_runs.get('data'):
                        deployed_at_str = bot_runs['data'][0].get('deployed_at')
                        if deployed_at_str:
                            bot_deployed_at = datetime.fromisoformat(deployed_at_str.replace('Z', '+00:00'))
                            print(f"   Deployed: {bot_deployed_at.strftime('%Y-%m-%d %H:%M UTC')}")

                    # Get controller configs
                    controller_configs_list = await client.controllers.get_bot_controller_configs(
                        bot_name=running_bot_name
                    )

                    # Convert to dict
                    controller_configs = {}
                    if isinstance(controller_configs_list, list):
                        for config in controller_configs_list:
                            if isinstance(config, dict) and 'id' in config:
                                controller_configs[config['id']] = config

                    # Parse controllers
                    performance = bot_info.get('performance', {})

                    for ctrl_id, ctrl_perf in performance.items():
                        if isinstance(ctrl_perf, dict) and 'performance' in ctrl_perf:
                            perf_data = ctrl_perf['performance']
                            ctrl_config = controller_configs.get(ctrl_id, {})

                            trading_pair = ctrl_config.get('trading_pair', 'UNKNOWN')
                            side = ctrl_config.get('side', 0)
                            direction = 'long' if side == 1 else 'short' if side == 2 else 'unknown'

                            runtime_hours = 0
                            if bot_deployed_at:
                                runtime = datetime.now(timezone.utc) - bot_deployed_at
                                runtime_hours = runtime.total_seconds() / 3600

                            bot_controllers.append({
                                'controller_id': ctrl_id,
                                'trading_pair': trading_pair,
                                'direction': direction,
                                'status': ctrl_perf.get('status', 'unknown'),
                                'pnl_quote': perf_data.get('global_pnl_quote', 0),
                                'pnl_pct': perf_data.get('global_pnl_pct', 0),
                                'volume': perf_data.get('volume_traded', 0),
                                'runtime_hours': runtime_hours,
                                'manual_kill_switch': ctrl_config.get('manual_kill_switch', False),
                                'config': ctrl_config
                            })
                else:
                    print(f"ℹ️  No active bot found with name '{CONFIG['bot_name']}'")
                    print("   Will deploy fresh instance")

    except Exception as e:
        print(f"❌ Error fetching bot status: {e}")
        import traceback
        traceback.print_exc()

# Display bot status
if bot_controllers:
    total_pnl = sum(ctrl['pnl_quote'] for ctrl in bot_controllers)
    total_volume = sum(ctrl['volume'] for ctrl in bot_controllers)
    winners = [c for c in bot_controllers if c['pnl_quote'] > 0]
    losers = [c for c in bot_controllers if c['pnl_quote'] < 0]
    runtime_hours = bot_controllers[0]['runtime_hours']
    
    print("\n" + "=" * 80)
    print(f"🤖 BOT: {running_bot_name}")
    print("=" * 80)
    print(f"Status: ✅ Running | Runtime: {runtime_hours:.1f}h | Min Runtime: {CONFIG['min_runtime_hours']}h")
    print(f"Controllers: {len(bot_controllers)} | Winners: {len(winners)} | Losers: {len(losers)}")
    print(f"Total P&L: ${total_pnl:+.2f} | Volume: ${total_volume:,.0f}")
    
    if winners:
        print(f"\n🎯 WINNERS ({len(winners)}):")
        for ctrl in sorted(winners, key=lambda x: x['pnl_quote'], reverse=True):
            dir_emoji = "🟢" if ctrl['direction'] == 'long' else "🔴"
            print(f"  {dir_emoji} {ctrl['trading_pair']:12s} {ctrl['direction']:5s} | "
                  f"${ctrl['pnl_quote']:+7.2f} ({ctrl['pnl_pct']:+7.2f}%) | Vol: ${ctrl['volume']:>8,.0f}")
    
    if losers:
        print(f"\n📉 LOSERS ({len(losers)}):")
        for ctrl in sorted(losers, key=lambda x: x['pnl_quote']):
            dir_emoji = "🟢" if ctrl['direction'] == 'long' else "🔴"
            print(f"  {dir_emoji} {ctrl['trading_pair']:12s} {ctrl['direction']:5s} | "
                  f"${ctrl['pnl_quote']:+7.2f} ({ctrl['pnl_pct']:+7.2f}%) | Vol: ${ctrl['volume']:>8,.0f}")
    
    print("\n" + "=" * 80)
    print(f"💰 TOTAL P&L: ${total_pnl:+.2f}")
    print("=" * 80)
    
    # Send Telegram notification about bot status
    if CONFIG['send_telegram_notifications']:
        msg = f"""<b>🤖 Bot Status: {running_bot_name}</b>

<b>⏱ Runtime:</b> {runtime_hours:.1f}h
<b>📊 Controllers:</b> {len(bot_controllers)} active
<b>💰 Total P&L:</b> <code>${total_pnl:+.2f}</code> ({(total_pnl/float(CONFIG['total_amount_quote'])/len(bot_controllers)*100) if bot_controllers else 0:+.2f}%)
<b>📈 Volume:</b> ${total_volume:,.0f}

<b>🎯 Winners:</b> {len(winners)} | <b>📉 Losers:</b> {len(losers)}
"""
        notification = NotificationMessage(
            title="🤖 Current Bot Status",
            message=msg,
            level="info"
        )
        await notification_manager.send_notification(notification)
        print("\n✅ Bot status sent to Telegram")
else:
    print(f"\n✅ No active bot - ready to deploy {CONFIG['bot_name']}")

🤖 Fetching active bot status from Hummingbot API...

✅ Found active bot: trend_follower_grid-20251014-135851
   Deployed: 2025-10-14 16:58 UTC

🤖 BOT: trend_follower_grid-20251014-135851
Status: ✅ Running | Runtime: 5.4h | Min Runtime: 4h
Controllers: 10 | Winners: 6 | Losers: 3
Total P&L: $+12.92 | Volume: $11,381

🎯 WINNERS (6):
  🟢 TIA-USDT     long  | $  +9.00 (  +0.43%) | Vol: $   2,105
  🔴 SQD-USDT     short | $  +6.40 (  +0.50%) | Vol: $   1,285
  🟢 ALGO-USDT    long  | $  +5.51 (  +0.41%) | Vol: $   1,334
  🟢 BEAMX-USDT   long  | $  +5.42 (  +0.49%) | Vol: $   1,104
  🟢 MTL-USDT     long  | $  +3.53 (  +0.40%) | Vol: $     887
  🟢 XTZ-USDT     long  | $  +2.75 (  +0.28%) | Vol: $     968

📉 LOSERS (3):
  🔴 BAT-USDT     short | $ -13.14 (  -1.44%) | Vol: $     912
  🔴 PTB-USDT     short | $  -5.34 (  -0.35%) | Vol: $   1,538
  🔴 STO-USDT     short | $  -1.23 (  -0.10%) | Vol: $   1,248

💰 TOTAL P&L: $+12.92

✅ Bot status sent to Telegram


## 3. Analyze Controllers - Decide What to Stop/Keep

In [39]:
# Analyze each controller and decide action
controllers_to_stop = []
controllers_to_keep = []
stop_reasons = {}

min_runtime_seconds = CONFIG['min_runtime_hours'] * 3600

print("🔍 Analyzing controllers...\n")

for ctrl in bot_controllers:
    ctrl_id = ctrl['controller_id']
    trading_pair = ctrl['trading_pair']
    direction = ctrl['direction']
    pnl_pct = ctrl['pnl_pct']
    runtime_hours = ctrl['runtime_hours']
    
    should_stop = False
    reason = None
    
    # Check if already manually stopped
    if ctrl['manual_kill_switch']:
        reason = "Already stopped (manual_kill_switch=True)"
        controllers_to_keep.append(ctrl)
        print(f"  ⏸️  {trading_pair:12s} {direction:5s} - {reason}")
        continue
    
    # Check minimum runtime
    within_min_runtime = runtime_hours < CONFIG['min_runtime_hours']
    
    # Check signal alignment
    current_signal = signal_map.get(trading_pair)
    has_opposing_signal = False
    
    if current_signal:
        # Check if signal direction opposes controller direction
        if (direction == 'long' and current_signal['direction'] == 'short') or \
           (direction == 'short' and current_signal['direction'] == 'long'):
            has_opposing_signal = True
    
    # Decision logic
    if within_min_runtime:
        # Within minimum runtime - keep regardless of P&L or signal
        controllers_to_keep.append(ctrl)
        print(f"  ✅ {trading_pair:12s} {direction:5s} - Keep (runtime {runtime_hours:.1f}h < {CONFIG['min_runtime_hours']}h min) | P&L: ${ctrl['pnl_quote']:+.2f}")
    else:
        # Past minimum runtime - check stop conditions
        if CONFIG['stop_on_opposing_signal'] and has_opposing_signal:
            should_stop = True
            reason = f"Opposing signal (running {direction}, signal is {current_signal['direction']})"
        elif pnl_pct < CONFIG['min_pnl_pct_to_keep']:
            should_stop = True
            reason = f"Poor P&L ({pnl_pct:.2f}% < {CONFIG['min_pnl_pct_to_keep']}%)"
        else:
            controllers_to_keep.append(ctrl)
            signal_status = "✨ matching signal" if current_signal and current_signal['direction'] == direction else "no signal"
            print(f"  ✅ {trading_pair:12s} {direction:5s} - Keep ({signal_status}) | P&L: ${ctrl['pnl_quote']:+.2f} ({pnl_pct:+.2f}%)")
    
    if should_stop:
        controllers_to_stop.append(ctrl)
        stop_reasons[ctrl_id] = reason
        print(f"  🛑 {trading_pair:12s} {direction:5s} - STOP: {reason} | P&L: ${ctrl['pnl_quote']:+.2f}")

print(f"\n📊 Analysis Summary:")
print(f"  Controllers to keep: {len(controllers_to_keep)}")
print(f"  Controllers to stop: {len(controllers_to_stop)}")

🔍 Analyzing controllers...

  ✅ BEAMX-USDT   long  - Keep (✨ matching signal) | P&L: $+5.42 (+0.49%)
  ✅ XTZ-USDT     long  - Keep (✨ matching signal) | P&L: $+2.75 (+0.28%)
  ✅ ALGO-USDT    long  - Keep (✨ matching signal) | P&L: $+5.51 (+0.41%)
  ✅ MTL-USDT     long  - Keep (✨ matching signal) | P&L: $+3.53 (+0.40%)
  ✅ TIA-USDT     long  - Keep (✨ matching signal) | P&L: $+9.00 (+0.43%)
  ✅ TWT-USDT     short - Keep (✨ matching signal) | P&L: $+0.00 (+0.00%)
  ✅ SQD-USDT     short - Keep (✨ matching signal) | P&L: $+6.40 (+0.50%)
  ✅ STO-USDT     short - Keep (✨ matching signal) | P&L: $-1.23 (-0.10%)
  🛑 PTB-USDT     short - STOP: Poor P&L (-0.35% < -0.3%) | P&L: $-5.34
  🛑 BAT-USDT     short - STOP: Poor P&L (-1.44% < -0.3%) | P&L: $-13.14

📊 Analysis Summary:
  Controllers to keep: 8
  Controllers to stop: 2


## 4. Stop Controllers

In [30]:
    # Stop controllers that should be stopped
stopped_controllers = []

if controllers_to_stop and CONFIG['auto_stop_controllers'] and running_bot_name:
    print(f"🛑 Stopping {len(controllers_to_stop)} controllers...\n")
    
    async with HummingbotAPIClient(base_url=f"http://{CONFIG['hummingbot_host']}:8000") as client:
        for ctrl in controllers_to_stop:
            ctrl_id = ctrl['controller_id']
            reason = stop_reasons.get(ctrl_id, 'Unknown')
            
            try:
                # Update controller config to set manual_kill_switch = True
                await client.controllers.update_bot_controller_config(
                    bot_name=running_bot_name,
                    controller_name=ctrl_id,
                    config={'manual_kill_switch': True}
                )
                
                stopped_controllers.append(ctrl)
                print(f"  ✅ Stopped {ctrl['trading_pair']:12s} {ctrl['direction']:5s} - {reason}")
                
            except Exception as e:
                print(f"  ❌ Failed to stop {ctrl['trading_pair']}: {e}")
    
    print(f"\n✅ Successfully stopped {len(stopped_controllers)} controllers")
elif controllers_to_stop:
    print(f"⚠️  Auto-stop is disabled. Would stop {len(controllers_to_stop)} controllers:")
    for ctrl in controllers_to_stop:
        print(f"  - {ctrl['trading_pair']} {ctrl['direction']}: {stop_reasons.get(ctrl['controller_id'])}")
else:
    print("ℹ️  No controllers to stop")

ℹ️  No controllers to stop


## 5. Determine New Controllers to Deploy

In [31]:
# Helper function to calculate grid levels
async def calculate_grid_levels(trading_pair, connector_name, feature_name, direction='long'):
    features = await storage.get_features(
        feature_name=feature_name,
        trading_pair=trading_pair,
        connector_name=connector_name,
        limit=1
    )
    
    if not features:
        raise ValueError(f"No feature found for {trading_pair}")
    
    feat = features[0]
    price = feat.value['price']
    range_pct = feat.value['range_pct']
    multipliers = CONFIG['grid_range_multiplier'][direction]
    
    return {
        'current_price': price,
        'range_pct': range_pct,
        'start_price': price * (1 + multipliers['start_offset'] * range_pct),
        'end_price': price * (1 + multipliers['end_offset'] * range_pct),
        'limit_price': price * (1 + multipliers['limit_offset'] * range_pct),
        'feature_timestamp': feat.timestamp
    }

def generate_grid_config(trading_pair, side, signal_value, grid_levels):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    direction = "long" if side == 1 else "short"
    config_id = f"tf_{trading_pair.lower().replace('-', '_')}_{direction}_{timestamp}"

    return {
        'id': config_id,
        'controller_name': 'grid_strike',
        'controller_type': 'generic',
        'connector_name': CONFIG['connector_name'],
        'trading_pair': trading_pair,
        'side': side,
        'position_mode': CONFIG['position_mode'],
        'leverage': CONFIG['leverage'],
        'start_price': str(grid_levels['start_price']),
        'end_price': str(grid_levels['end_price']),
        'limit_price': str(grid_levels['limit_price']),
        'total_amount_quote': CONFIG['total_amount_quote'],
        'max_open_orders': CONFIG['max_open_orders'],
        'max_orders_per_batch': 2,
        'min_order_amount_quote': '6',
        'min_spread_between_orders': CONFIG['min_spread_between_orders'],
        'activation_bounds': CONFIG['activation_bounds'],
        'order_frequency': 5,
        'keep_position': False,
        'triple_barrier_config': {
            'open_order_type': 3,
            'take_profit': CONFIG['take_profit'],
            'take_profit_order_type': 3
        },
        'signal_value': signal_value,
    }

# Determine how many controllers we can deploy
current_active_controllers = len(controllers_to_keep)
max_new_controllers = CONFIG['max_controllers_per_instance'] - current_active_controllers

print(f"📊 Controller Capacity:")
print(f"  Controllers to keep: {len(controllers_to_keep)}")
print(f"  Controllers to stop: {len(stopped_controllers)}")
print(f"  Available slots: {max_new_controllers}\n")

# Get markets that are ACTUALLY staying active (excluding stopped ones)
running_markets = {ctrl['trading_pair']: ctrl['direction'] for ctrl in controllers_to_keep}
stopped_markets = {ctrl['trading_pair']: ctrl['direction'] for ctrl in stopped_controllers}

# Identify new opportunities
new_configs = []
new_long_signals = [sig for sig in long_signals[:CONFIG['top_n_per_side']]]
new_short_signals = [sig for sig in short_signals[:CONFIG['top_n_per_side']]]

print("🔍 Analyzing opportunities for new controllers...\n")

# Process long signals
for signal in new_long_signals:
    if len(new_configs) >= max_new_controllers:
        break
    
    trading_pair = signal.trading_pair
    
    # Skip if already running in same direction (keeping it)
    if trading_pair in running_markets and running_markets[trading_pair] == 'long':
        print(f"  ⏭️  {trading_pair:12s} LONG  - Already running, keeping")
        continue
    
    # Skip if just stopped - don't redeploy immediately
    if trading_pair in stopped_markets and stopped_markets[trading_pair] == 'long':
        print(f"  ⛔ {trading_pair:12s} LONG  - Just stopped, skipping")
        continue
    
    # Generate config for new controller
    try:
        grid_levels = await calculate_grid_levels(
            trading_pair, CONFIG['connector_name'], CONFIG['feature_name'], 'long'
        )
        config = generate_grid_config(trading_pair, 1, signal.value, grid_levels)
        new_configs.append(config)
        print(f"  🆕 {trading_pair:12s} LONG  - New opportunity | Signal: {signal.value:+.3f}")
    except Exception as e:
        print(f"  ❌ {trading_pair} LONG: {e}")

# Process short signals
for signal in new_short_signals:
    if len(new_configs) >= max_new_controllers:
        break
    
    trading_pair = signal.trading_pair
    
    # Skip if already running in same direction (keeping it)
    if trading_pair in running_markets and running_markets[trading_pair] == 'short':
        print(f"  ⏭️  {trading_pair:12s} SHORT - Already running, keeping")
        continue
    
    # Skip if just stopped - don't redeploy immediately
    if trading_pair in stopped_markets and stopped_markets[trading_pair] == 'short':
        print(f"  ⛔ {trading_pair:12s} SHORT - Just stopped, skipping")
        continue
    
    # Generate config for new controller
    try:
        grid_levels = await calculate_grid_levels(
            trading_pair, CONFIG['connector_name'], CONFIG['feature_name'], 'short'
        )
        config = generate_grid_config(trading_pair, 2, signal.value, grid_levels)
        new_configs.append(config)
        print(f"  🆕 {trading_pair:12s} SHORT - New opportunity | Signal: {signal.value:+.3f}")
    except Exception as e:
        print(f"  ❌ {trading_pair} SHORT: {e}")

print(f"\n✅ Identified {len(new_configs)} new controllers to deploy")

if stopped_controllers and not new_configs:
    print(f"\n⚠️  Note: Stopped {len(stopped_controllers)} controllers but no new opportunities available")
    print("   Consider expanding signal search or adjusting min intensity thresholds")

📊 Controller Capacity:
  Controllers to keep: 10
  Controllers to stop: 0
  Available slots: 0

🔍 Analyzing opportunities for new controllers...


✅ Identified 0 new controllers to deploy


## 6. Deploy New Controllers

In [23]:
# Deploy new controllers - SIMPLIFIED
deployed_configs = []

if new_configs and CONFIG['auto_deploy']:
    print(f"🚀 Deploying {len(new_configs)} new controllers...\n")
    
    async with HummingbotAPIClient(base_url=f"http://{CONFIG['hummingbot_host']}:8000") as client:
        # Create controller configs
        for config in new_configs:
            try:
                api_config = {k: v for k, v in config.items() if k not in ['signal_value']}
                
                await client.controllers.create_or_update_controller_config(
                    config_name=config['id'],
                    config=api_config
                )
                
                deployed_configs.append(config)
                direction = "LONG" if config['side'] == 1 else "SHORT"
                print(f"  ✅ Created config: {config['trading_pair']:12s} {direction}")
                
            except Exception as e:
                print(f"  ❌ Failed to create config for {config['trading_pair']}: {e}")
        
        # Deploy bot with configs
        if deployed_configs:
            try:
                await client.bot_orchestration.deploy_v2_controllers(
                    instance_name=CONFIG['bot_name'],  # Use bot_name directly
                    credentials_profile=CONFIG['credentials_profile'],
                    controllers_config=[c['id'] for c in deployed_configs]
                )
                print(f"\n✅ Successfully deployed bot '{CONFIG['bot_name']}' with {len(deployed_configs)} controllers")
                
                # Send detailed Telegram notification
                if CONFIG['send_telegram_notifications']:
                    msg = f"""<b>🚀 Deployment Complete</b>

<b>Bot Name:</b> <code>{CONFIG['bot_name']}</code>
<b>Controllers:</b> {len(deployed_configs)}
<b>Total Capital:</b> ${float(CONFIG['total_amount_quote']) * len(deployed_configs):,.0f}

<b>🟢 Long Positions ({sum(1 for c in deployed_configs if c['side'] == 1)})</b>
"""
                    for config in [c for c in deployed_configs if c['side'] == 1]:
                        msg += f"• <code>{config['trading_pair']:12s}</code> Signal: {config['signal_value']:+.3f}\n"
                    
                    msg += f"\n<b>🔴 Short Positions ({sum(1 for c in deployed_configs if c['side'] == 2)})</b>\n"
                    for config in [c for c in deployed_configs if c['side'] == 2]:
                        msg += f"• <code>{config['trading_pair']:12s}</code> Signal: {config['signal_value']:+.3f}\n"
                    
                    msg += f"\n<b>⚙️ Settings</b>\n"
                    msg += f"• Leverage: {CONFIG['leverage']}x\n"
                    msg += f"• Amount per grid: ${CONFIG['total_amount_quote']}\n"
                    msg += f"• Take Profit: {float(CONFIG['take_profit'])*100:.2f}%\n"
                    
                    notification = NotificationMessage(
                        title="🚀 Bot Deployed",
                        message=msg,
                        level="info"
                    )
                    await notification_manager.send_notification(notification)
                    print("✅ Deployment notification sent to Telegram")
                    
            except Exception as e:
                print(f"❌ Failed to deploy bot: {e}")
                
elif new_configs:
    print(f"⚠️  Auto-deploy disabled. Would deploy {len(new_configs)} controllers")
else:
    print("ℹ️  No new controllers to deploy")

ℹ️  No new controllers to deploy


## 7. Send Telegram Report

In [24]:
# Final Summary Report via Telegram
if CONFIG['send_telegram_notifications']:
    total_active = len(controllers_to_keep) + len(deployed_configs)
    
    telegram_message = f"""<b>🎉 Bot Orchestration Complete</b>
━━━━━━━━━━━━━━━━━━━━━━━━━━━━

<b>📊 Summary</b>
├ Bot: <code>{CONFIG['bot_name']}</code>
├ Active Controllers: <code>{total_active}/{CONFIG['max_controllers_per_instance']}</code>
├ Stopped: <code>{len(stopped_controllers)}</code>
└ Deployed: <code>{len(deployed_configs)}</code>

"""

    if stopped_controllers:
        telegram_message += "<b>🛑 Stopped Controllers</b>\n"
        for ctrl in stopped_controllers:
            dir_emoji = "🟢" if ctrl['direction'] == 'long' else "🔴"
            reason = stop_reasons.get(ctrl['controller_id'], 'Unknown')
            telegram_message += f"{dir_emoji} <code>{ctrl['trading_pair']:12s}</code> ${ctrl['pnl_quote']:+.2f} - {reason}\n"
        telegram_message += "\n"

    if deployed_configs:
        telegram_message += f"<b>🆕 Deployed ({len(deployed_configs)})</b>\n"
        for config in deployed_configs:
            dir_emoji = "🟢" if config['side'] == 1 else "🔴"
            direction = "LONG" if config['side'] == 1 else "SHORT"
            telegram_message += f"{dir_emoji} <code>{config['trading_pair']:12s}</code> {direction:5s} | Signal: {config['signal_value']:+.3f}\n"
        telegram_message += "\n"
    
    if controllers_to_keep:
        telegram_message += f"<b>✅ Keeping ({len(controllers_to_keep)})</b>\n"
        total_pnl = sum(ctrl['pnl_quote'] for ctrl in controllers_to_keep)
        telegram_message += f"Total P&L: <code>${total_pnl:+.2f}</code>\n\n"
    
    telegram_message += f"""<b>⚙️ Management Rules</b>
├ Min Runtime: <code>{CONFIG['min_runtime_hours']}h</code>
├ Min P&L: <code>{CONFIG['min_pnl_pct_to_keep']}%</code>
└ Stop on Signal Change: <code>{CONFIG['stop_on_opposing_signal']}</code>

<i>📅 {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S UTC')}</i>"""

    notification = NotificationMessage(
        title="🎉 Orchestration Complete",
        message=telegram_message,
        level="info",
    )

    print("📤 Sending final report via Telegram...\n")
    
    try:
        results = await notification_manager.send_notification(notification)
        
        for service, success in results.items():
            status = "✅ Sent" if success else "❌ Failed"
            print(f"  {service.capitalize()}: {status}")
        
        if results.get('telegram'):
            print("\n🎉 Final report sent to Telegram!")
            
    except Exception as e:
        print(f"❌ Error sending notification: {e}")
else:
    print("ℹ️  Telegram notifications disabled")

📤 Sending final report via Telegram...

  Telegram: ✅ Sent

🎉 Final report sent to Telegram!


## Summary

In [ ]:
print("\n" + "="*80)
print("🎉 SMART BOT ORCHESTRATION COMPLETE")
print("="*80)
print(f"\n📊 Final State:")
print(f"  Bot Name: {CONFIG['bot_name']}")
print(f"  Active Controllers: {len(controllers_to_keep) + len(deployed_configs)}")
print(f"  Stopped Controllers: {len(stopped_controllers)}")
print(f"  New Controllers Deployed: {len(deployed_configs)}")

if controllers_to_keep:
    total_pnl = sum(ctrl['pnl_quote'] for ctrl in controllers_to_keep)
    print(f"\n💰 Active Controllers P&L: ${total_pnl:+.2f}")

if deployed_configs:
    total_capital = float(CONFIG['total_amount_quote']) * len(deployed_configs)
    print(f"💵 Newly Deployed Capital: ${total_capital:,.0f}")

print("\n" + "="*80)